# Task 1

In [78]:
def my_own_fasta_parser(inFile):

    sequences = {}

    with open(inFile) as file:
        for line in file.readlines():
            line = line.strip() 
            if line[0] == '>': 
                seq_id = line[1:] 
            else:
                sequences[seq_id] = line 

    return sequences


In [79]:
my_own_fasta_parser('prot.fasta')

{'seq0': 'FQTWEEFSRAAEKLYLADPMKVRVVLKYRHVDGNLCIKVTDDLVCLVYRTDQAQDVKKIEKF',
 'seq1': 'KYRTWEEFTRAAEKLYQADPMKVRVVLKYRHCDGNLCIKVTDDVVCLLYRTDQAQDVKKIEKFHSQLMRLME LKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq2': 'EEYQTWEEFARAAEKLYLTDPMKVRVVLKYRHCDGNLCMKVTDDAVCLQYKTDQAQDVKKVEKLHGK',
 'seq3': 'MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDVK',
 'seq4': 'EEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVVSYEMRLFGVQKDNFALEHSLL',
 'seq5': 'SWEEFAKAAEVLYLEDPMKCRMCTKYRHVDHKLVVKLTDNHTVLKYVTDMAQDVKKIEKLTTLLMR',
 'seq6': 'FTNWEEFAKAAERLHSANPEKCRFVTKYNHTKGELVLKLTDDVVCLQYSTNQLQDVKKLEKLSSTLLRSI',
 'seq7': 'SWEEFVERSVQLFRGDPNATRYVMKYRHCEGKLVLKVTDDRECLKFKTDQAQDAKKMEKLNNIFF',
 'seq8': 'SWDEFVDRSVQLFRADPESTRYVMKYRHCDGKLVLKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq9': 'KNWEDFEIAAENMYMANPQNCRYTMKYVHSKGHILLKMSDNVKCVQYRAENMPDLKK',
 'seq10': 'FDSWDEFVSKSVELFRNHPDTTRYVVKYRHCEGKLVLKVTDNHECLKFKTDQAQDAKKMEK'}

# Task 2

In [80]:
def my_own_residue_abundance(input_file, residue, threshold = 0.2):
    
    seq_ids = []
    sequences = my_own_fasta_parser(input_file)

    for seq_id, sequence in sequences.items():
        freq = sequence.count(residue) / len(sequence)
        if freq > threshold:
            seq_ids.append(seq_id)

    return seq_ids

In [81]:
print(my_own_residue_abundance('prot.fasta', 'L', 0.1))
print(my_own_residue_abundance('prot.fasta', 'L', 0.05))
print(my_own_residue_abundance('prot.fasta', 'Q', 0.05))

['seq1', 'seq4', 'seq5', 'seq6']
['seq0', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6', 'seq7', 'seq8', 'seq9', 'seq10']
['seq1', 'seq2', 'seq3']


# Task 3

In [82]:
import Bio
from Bio.Seq import Seq
from Bio import SeqIO, SeqUtils 
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqRecord import SeqRecord


In [83]:

def my_own_filtering(input_file, output_file, filt_gc = 45, filt_arom = 0.125):
    
    sequences = {}
    c = 0
    
    with open(input_file, "r") as content:
        
        for record in SeqIO.parse(content, "fasta"):
            c += 1
            
            # calculate GC content using Bio
            calc_gc = SeqUtils.GC(record.seq)
            
            # calculate aromaticity using Bio
            protein = record.seq.translate()
            calc_arom = ProteinAnalysis(str(protein)).aromaticity()
            
            # so, now you can filter
            if calc_gc >= filt_gc and calc_arom >= filt_arom:
                sequences[record.id] = record.seq
    
    # write a new fasta file with aminoacids
    records = []
    for seq_id, seq in sequences.items():
        records.append(SeqRecord(seq, seq_id))
    with open(output_file, 'w+') as f:
        SeqIO.write(records, f, 'fasta')

    # print the percentage
    print("%0.1f" % (100 * len(sequences) / c) + '% of seqs passed the filter')

In [84]:

my_own_filtering('nucl.fasta', 'out.fasta')


14.3% of seqs passed the filter


# Task 4

In [85]:
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist

In [86]:
def balign(first_seq, second_seq, gap_open = -10, gap_extend = -0.5):

    # Load the matrix
    matrix = matlist.blosum62

    # Generate the alignments
    alns = pairwise2.align.globalds(first_seq, second_seq, matrix, gap_open, gap_extend, penalize_end_gaps=(False, False))

    # Extract the best alignment (first one in the alns list)
    top_aln = alns[0]

    # Print the alignment, ...
    aln_A, aln_B, score, begin, end = top_aln
    print(pairwise2.format_alignment(aln_A, aln_B, score, begin, end))

In [87]:
balign("AAATGCTACTG", "ATCGT")
balign("AAATGCTACTG", "AAA")
balign("AAATGCTACTG", "TTT")

AAATGCTACTG-
       |..| 
-------ATCGT
  Score=8

AAATGCTACTG
|||        
AAA--------
  Score=12

AAATGCTACTG
 ..|       
-TTT-------
  Score=5



# Task 5

In [100]:

def get_peptides(dna_seq):
    # translating from both coding DNA and reverse complement DNA, 
    # while taking stop codons into account
    coding_dna = Seq(dna_seq)
    compl_dna = coding_dna.reverse_complement()
    peptides = []
    for i in range(len(coding_dna)):
        peptide = str(coding_dna[i:].translate(to_stop = True))
        if len(peptide) > 1:
            peptides.append(peptide)
    for i in range(len(compl_dna)):
        peptide = str(compl_dna[i:].translate(to_stop = True))
        if len(peptide) > 1:
            peptides.append(peptide)
    for peptide in sorted(peptides, key = len):
        print(peptide)
    

In [101]:

dna_seq = 'AGTACTAGAGCATTCTATGGAG'
get_peptides(dna_seq)


LW
YG
ME
ML
LV
ST
ILW
FYG
SME
RML
ALV
SST
SILW
AFYG
HSME
HRML
NALV
CSST
RAFYG
EHSME
LHRML
ECSST
TRAFYG
LEHSME
IECSST
STRAFYG
VLEHSME
SIECSST


# Task 6

In [90]:
def rev_compl_one_line(seq):
    return "".join(reversed([{'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}[char] for char in seq]))

In [91]:
print(rev_compl_one_line('TGCAA'))
print(rev_compl_one_line('AGCTTCAGCTACGT'))

TTGCA
ACGTAGCTGAAGCT
